## 1. Importing data and data preprocessing

### 1.1 Import necesary libraries

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
# import matplotlib.pyplot as plt
import json
import random
import tensorflowjs as tfjs

### 1.2 Load and preprocess data

In [2]:
# Prepare training data
training_data = []

# Hold training
with open('hold_training.json') as json_file:  
    data = json.load(json_file)
    for coordSet in data['coords']:
        training_data.append([coordSet['mousePos'],0]) # 0 is hold
        
# Spill training
with open('spill_training.json') as json_file:  
    data = json.load(json_file)
    for coordSet in data['coords']:
        # Convert and append mousePosArr
        training_data.append([coordSet['mousePos'],1]) # 1 is spill

# Shuffling
random.shuffle(training_data)

# Extract
train_coords = []
train_labels = []
minVal = float('inf')
maxVal = 0
for sample in training_data:
    train_coords.append(sample[0])
    train_labels.append(sample[1])
    minVal = min(minVal, min(sample[0]))
    maxVal = max(maxVal, max(sample[0]))
# Normalize
difVal = maxVal - minVal
for i in range(len(train_coords)):
    for j in range(len(train_coords[i])):
        train_coords[i][j] = (train_coords[i][j] - minVal) / float(difVal)
    # Convert to np array
    train_coords[i] = np.asarray(train_coords[i])

# Convert
train_coords = np.asarray(train_coords)
train_labels = np.asarray(train_labels)
print(train_coords)
print(train_labels)

[[0.2994527  0.00547303 0.2994527  ... 0.06176701 0.43784206 0.06176701]
 [0.48084441 0.33854574 0.48084441 ... 0.13760751 0.48006255 0.13760751]
 [0.83580923 0.10476935 0.83580923 ... 0.12666145 0.54573886 0.10007819]
 ...
 [0.97107115 0.15011728 0.97107115 ... 0.06567631 0.56919468 0.07114934]
 [0.60125098 0.00703675 0.60125098 ... 0.06098514 0.50195465 0.06176701]
 [0.30101642 0.34089132 0.30101642 ... 0.14073495 0.45895231 0.13682565]]
[1 0 1 1 0 0 0 1 0 1 0 1 0 1 0 1 1 1 0 0]


In [3]:
# Prepare testing data
testing_data = []

# Hold testing
with open('hold_testing.json') as json_file:  
    data = json.load(json_file)
    for coordSet in data['coords']:
        testing_data.append([coordSet['mousePos'],0]) # 0 is hold
        
# Spill training
with open('spill_testing.json') as json_file:  
    data = json.load(json_file)
    for coordSet in data['coords']:
        # Convert and append mousePosArr
        testing_data.append([coordSet['mousePos'],1]) # 1 is spill

# Shuffling
random.shuffle(testing_data)

# Extract
test_coords = []
test_labels = []
minVal = float('inf')
maxVal = 0
for sample in testing_data:
    test_coords.append(sample[0])
    test_labels.append(sample[1])
    minVal = min(minVal, min(sample[0]))
    maxVal = max(maxVal, max(sample[0]))
# Normalize
difVal = maxVal - minVal
for i in range(len(test_coords)):
    for j in range(len(test_coords[i])):
        test_coords[i][j] = (test_coords[i][j] - minVal) / float(difVal)
    # Convert to np array
    test_coords[i] = np.asarray(test_coords[i])

# Convert
test_coords = np.asarray(test_coords)
test_labels = np.asarray(test_labels)
print(test_coords)
print(test_labels)

[[0.25929127 0.07260156 0.25929127 ... 0.08556612 0.4684529  0.08643042]
 [0.80207433 0.17199654 0.80207433 ... 0.1227312  0.59982714 0.12100259]
 [0.35609334 0.01037165 0.35609334 ... 0.07778738 0.50648228 0.07778738]
 ...
 [0.83837511 0.1426102  0.83837511 ... 0.1642178  0.51512532 0.1642178 ]
 [0.94122731 0.15816768 0.94122731 ... 0.17286085 0.5911841  0.17286085]
 [0.21866897 0.07778738 0.21866897 ... 0.10025929 0.43301642 0.10025929]]
[1 0 0 1 0 0 1 1 0 1 1 0 0 0 0 1 0 1 1 1]


## 2. CNN

### 2.1 Build model

In [4]:
model = keras.Sequential([
#     keras.layers.Flatten(input_shape=(1,300)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(2, activation=tf.nn.softmax)
])

### 2.2 Compile 

In [5]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### 2.3 Train

In [6]:
model.fit(train_coords, train_labels, epochs=50)

W0717 09:42:40.266690 4486510016 deprecation.py:323] From /Users/tgoh/Documents/sippy-cup/env/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
20/20 [==============================] - 0s 3ms/sample - loss: 0.7095 - acc: 0.5500
Epoch 2/50
20/20 [==============================] - 0s 73us/sample - loss: 0.6664 - acc: 0.7500
Epoch 3/50
20/20 [==============================] - 0s 71us/sample - loss: 0.6402 - acc: 0.5000
Epoch 4/50
20/20 [==============================] - 0s 70us/sample - loss: 0.5998 - acc: 0.8500
Epoch 5/50
20/20 [==============================] - 0s 62us/sample - loss: 0.5802 - acc: 0.8500
Epoch 6/50
20/20 [==============================] - 0s 84us/sample - loss: 0.5570 - acc: 0.9000
Epoch 7/50
20/20 [==============================] - 0s 73us/sample - loss: 0.5383 - acc: 0.8500
Epoch 8/50
20/20 [==============================] - 0s 99us/sample - loss: 0.5251 - acc: 0.8000
Epoch 9/50
20/20 [==============================] - 0s 76us/sample - loss: 0.5081 - acc: 0.8500
Epoch 10/50
20/20 [==============================] - 0s 93us/sample - loss: 0.4944 - acc: 0.9000
Epoch 11/50
20/20 [=====================

### 2.4 Results

In [7]:
test_loss, test_acc = model.evaluate(test_coords, test_labels)

20/20 [==============================] - 0s 926us/sample - loss: 0.5678 - acc: 0.6500


In [8]:
predictions = model.predict(test_coords)
print(predictions)

[[0.5495354  0.4504646 ]
 [0.79685175 0.20314825]
 [0.7893279  0.2106721 ]
 [0.56672955 0.43327048]
 [0.9652034  0.03479658]
 [0.62908685 0.37091315]
 [0.6171704  0.38282967]
 [0.01181204 0.98818797]
 [0.9289684  0.07103157]
 [0.36187607 0.6381239 ]
 [0.89591795 0.10408203]
 [0.95235723 0.04764277]
 [0.86622196 0.13377799]
 [0.94987434 0.0501257 ]
 [0.31448403 0.68551594]
 [0.14031948 0.85968053]
 [0.63492066 0.36507934]
 [0.6817806  0.31821942]
 [0.40717795 0.5928221 ]
 [0.73495054 0.26504946]]


### 2.5 Save model

In [12]:
# Python model
model.save('Model/model.h5')
# JS model
tfjs.converters.save_keras_model(model, './Model')